# PyKX Query Components
Query all TorQ Components.


In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import boto3
import json
import datetime

import pykx as kx

from managed_kx import *

from env import *
from config import *

GATEWAY='gateway1' # Cluster (gateway) that queries will be sent to

In [2]:
# Using credentials and create service client
session = boto3.Session()

# create finspace client
client = session.client(service_name='finspace')

# List of Clusters

In [3]:
cdf = get_clusters(client, environmentId=ENV_ID)

all_clusters = [d['name'] for d in clusters if 'name' in d]

if cdf is not None:
    cdf = cdf[cdf['clusterName'].isin(all_clusters)]
#    cdf = cdf[cdf['status'].isin(['RUNNING'])]

display(cdf)

,clusterName,status,clusterType,capacityConfiguration,commandLineArguments,clusterDescription,lastModifiedTimestamp,createdTimestamp,databaseName,cacheConfigurations
3,discovery1,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'discovery'}, {'key': 'procname', 'value': 'discovery1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '2'}]",Created with create_all notebook,2024-12-03 20:16:25.763000+00:00,2024-12-03 20:01:35.746000+00:00,finspace-database,None
4,feed1,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'tradeFeed'}, {'key': 'procname', 'value': 'tradeFeed1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '2'}]",Created with create_all notebook,2024-12-03 20:30:55.001000+00:00,2024-12-03 20:18:27.837000+00:00,finspace-database,None
5,gateway1,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'gateway'}, {'key': 'procname', 'value': 'gateway1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '2'}]",Created with create_all notebook,2024-12-03 20:30:55.616000+00:00,2024-12-03 20:18:24.649000+00:00,finspace-database,None
6,hdb1,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'hdb'}, {'key': 'procname', 'value': 'hdb1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '4'}]",Created with create_all notebook,2024-12-03 20:17:05.878000+00:00,2024-12-03 20:01:41.735000+00:00,finspace-database,None
7,monitor1,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'monitor'}, {'key': 'procname', 'value': 'monitor1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '1'}]",Created with create_all notebook,2024-12-03 20:29:55.787000+00:00,2024-12-03 20:18:30.995000+00:00,finspace-database,None
8,rdb1,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'rdb'}, {'key': 'procname', 'value': 'rdb1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '2'}]",Created with create_all notebook,2024-12-03 20:18:04.815000+00:00,2024-12-03 20:01:39.007000+00:00,finspace-database,None


# Query the system through the Gateway

In [4]:
conn = get_pykx_connection(client, 
    environmentId=ENV_ID, clusterName=GATEWAY, 
    userName=KDB_USERNAME, boto_session=session)

queries = [
    '.gw.syncexec["meta trades";`rdb]',
    '.gw.syncexec["meta trades";`hdb]',
    '.gw.syncexec["select sum size by sym from trades";`rdb]',
    '.gw.syncexec["count trades";`rdb]',
    '.gw.syncexec["select [5] from trades";`rdb]',
    '.gw.syncexec["select [-5] from trades";`rdb]',
    '.gw.syncexec["select count i by date from trades";`hdb]', # FIXED
    '.gw.syncexec["select sum size, max price by 0D00:05 xbar time from trades where sym=`IBM";`hdb`rdb]',
    '.gw.syncexecj["select sum size by sym from trades";`rdb`hdb;sum]',
]

for q in queries:
    print(100*"=")
    print(f"Query: {q}")
    print(100*"-")
    display( conn(q).pd() )

Query: .gw.syncexec["meta trades";`rdb]
----------------------------------------------------------------------------------------------------


,t,f,a
c,,,
sym,b's',,g
time,b'p',,
src,b's',,
price,b'f',,
size,b'i',,


Query: .gw.syncexec["meta trades";`hdb]
----------------------------------------------------------------------------------------------------


,t,f,a
c,,,
date,b'd',,
sym,b's',,p
time,b'p',,
src,b's',,
price,b'f',,
size,b'i',,


Query: .gw.syncexec["select sum size by sym from trades";`rdb]
----------------------------------------------------------------------------------------------------


...

,x
date,
2014-04-21,29229
2014-04-22,26305
2014-04-23,32186


Query: .gw.syncexec["select sum size, max price by 0D00:05 xbar time from trades where sym=`IBM";`hdb`rdb]
----------------------------------------------------------------------------------------------------


,size,price
time,,
2014-04-21 08:00:00,106759,41.18
2014-04-21 08:05:00,105275,41.31
2014-04-21 08:10:00,53978,41.18
2014-04-21 08:15:00,76514,41.29
2014-04-21 08:20:00,114316,41.03
...,...,...
2024-12-03 20:40:00,137914,43.67
2024-12-03 20:45:00,193739,43.63
2024-12-03 20:50:00,163706,48.88


Query: .gw.syncexecj["select sum size by sym from trades";`rdb`hdb;sum]
----------------------------------------------------------------------------------------------------


,size
sym,
AAPL,26086384
CSCO,26351004
DELL,26313317
GOOG,26651612
IBM,26626676
MSFT,27009623
NOK,26886988
ORCL,26593744
YHOO,26736205


In [5]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2024-12-03 21:03:21.125277
